In [1]:
# Installing the required package

%pip install python-arango

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 578.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing the package

from arango import ArangoClient

# Initialize the ArangoDB client

client = ArangoClient(hosts="http://127.0.0.1:8529")
db = client.db('_system', username='root', password='')

# AQL queries for edge creation
queries = [
    '''
    FOR c IN Customer
      FOR r IN Reservation
        FILTER r.Customer_ID == c.Customer_ID
        INSERT {
          "_from": c._id,
          "_to": r._id,
          "type": "booking"
        } INTO cust_resv
    ''',

    '''
    FOR r IN Reservation
      FOR s IN ShowTime
        FILTER r.Show_ID == s.Show_ID
        INSERT {
          "_from": r._id,
          "_to": s._id,
          "type": "forShowtime"
        } INTO res_show
    ''',

    '''
    FOR s IN ShowTime
      FOR m IN Movie
        FILTER s.Movie_ID == m.Movie_ID
        INSERT {
          "_from": s._id,
          "_to": m._id,
          "type": "forMovie"
        } INTO show_movie
    ''',

    '''
    FOR s IN ShowTime
        FOR t IN Theater
         FILTER s.Movie_ID == t.Movie_ID
            INSERT {
              "_from": s._id,
              "_to": t._id,
              "type": "forShowTheater"
            } INTO show_Theater
    ''',

    '''
    FOR m IN Movie
        FOR t IN Theater
            FILTER m.Movie_ID == t.Movie_ID
            INSERT {
              "_from": m._id,
              "_to": t._id,
              "type": "forMovieTheaters"
            } INTO Movie_Theater'''
]

# Executing the queries directly into the ArangoDB Web Interface
for query in queries:
  cursor = db.aql.execute(query)
  for doc in cursor:
    print(doc)
